In [ ]:
# Impor pustaka dasar
import matplotlib
import pandas as pd
import numpy as np
import seaborn as sns

# Impor pustaka visualisasi dan metrik
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import preprocessing, svm

# Impor pustaka classifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.naive_bayes import GaussianNB

# Itertools buat plotting
import itertools

In [ ]:
# Fungsi transpose kolom 'kelas' dari x menjadi y
def preprocess_data(x, channel):
    y = pd.read_excel(x)
    i = 0
    for i in range(len(y.index)):
        y.rename(index={i: channel[i]}, inplace=True)
        i+=1
    y = y.T
    y = y[1:]
    return y

In [ ]:
# Fungsi untuk mengaplikasikan data ke bilah kategori
def labels_to_categorical(x):
    y = to_categorical(x)
    return y

In [ ]:
# Fungsi normalisasi data dengan standar deviasi
def normalize_data(x):
    i = x.shape[1]
    i = i-1
    y = preprocessing.StandardScaler().fit(x).transform(x.astype(float))
    y = y[:,:i]
    return y

In [ ]:
# Fungsi untuk menjadikan kolom x sebuah array
def x_to_array(x):
    y = x.values
    return y

In [ ]:
# Fungsi untuk membuat kelas
def addclass(x, name):
    x['Class'] = pd.Series(name, index=x.index)
    return x

In [ ]:
# Fungsi untuk menggabungkan 5 array
def mergedf5(x1, x2, x3, x4, x5):
    y = pd.concat([x1, x2, x3, x4, x5])
    return y

In [ ]:
# Membuat grafis matrix kekalutan
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Reds):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    # Pembuatan matriks saat normalisasi 'true' atau 'false'
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')
    print(cm)
    
    # Pembuatan
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")
    # Label data
    plt.tight_layout()
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')

In [ ]:
# Pembuatan nama kelas
classes = ['60Co', '22Na', '241Am', 'Mix', '90Sr']

In [ ]:
classes = ['60Co', '22Na', '241Am', 'Mix', '90Sr']

In [ ]:
# Membuat indeks data
kal = pd.read_excel('Co_1.xlsx')
kal = kal.iloc[::,0]
kal = kal.astype(str)

In [ ]:
# Memproses semua data dan mengindekskan seperti input sebelumnya
co1 = preprocess_data('Co_1.xlsx', kal)
co2 = preprocess_data('Co_2.xlsx', kal)
na1 = preprocess_data('Na_1.xlsx', kal)
na2 = preprocess_data('Na_2.xlsx', kal)
am1 = preprocess_data('Am_1.xlsx', kal)
am2 = preprocess_data('Am_2.xlsx', kal)
mix1 = preprocess_data('Mix_1.xlsx', kal)
mix2 = preprocess_data('Mix_2.xlsx', kal)
sr1 = preprocess_data('Sr_1.xlsx', kal)
sr2 = preprocess_data('Sr_2.xlsx', kal)

# Test array length
print(co1.shape)

In [ ]:
# Menggabungkan data yang terpisah
co = pd.concat([co1, co2])
na = pd.concat([na1, na2])
am = pd.concat([am1, am2])
mix = pd.concat([mix1, mix2])
sr = pd.concat([sr1, sr2])

In [ ]:
# Memberikan indeks kelas pada masih-masing data
co = addclass(co, 0)
na = addclass(na, 1)
am = addclass(am, 2)
mix = addclass(mix, 3)
sr = addclass(sr, 4)

In [ ]:
all_data = mergedf5(co, na, am, mix, sr)

# Test length
all_data.shape

In [ ]:
testAm = preprocess_data('Test80_Am.xlsx', kal)
testCo = preprocess_data('Test80_Co.xlsx', kal)
testMix = preprocess_data('Test80_Mix.xlsx', kal)
testNa = preprocess_data('Test80_Na.xlsx', kal)
testSr = preprocess_data('Test80_Sr.xlsx', kal)

In [ ]:
# Memberikan indeks kelas pada masih-masing data
testCo = addclass(testCo, 0)
testNa = addclass(testNa, 1)
testAm = addclass(testAm, 2)
testMix = addclass(testMix, 3)
testSr = addclass(testSr, 4)

In [ ]:
test = mergedf5(testCo, testNa, testAm, testMix, testSr)

# Test Length
test.shape

In [ ]:
# Mengubah data training menjadi array dan menormalisasikannya
X = x_to_array(all_data)
X = normalize_data(X)

# Test Length
X.shape

In [ ]:
# Sama seperti sebelumnya, tapi ini kolom kelasnya doang
y = all_data['Class']
y = x_to_array(y)

# Test Length
y.shape

In [ ]:
# Masih sama, ini utuk data test
X_test = x_to_array(test)
X_test = normalize_data(X_test)

# Test Length
print(X_test.shape)

In [ ]:
y_test = test['Class']
y_test = x_to_array(y_test)

# Test Length
print(y_test.shape)

In [ ]:
# Memisahkan data training menjadi dua bagian untuk validasi
X_train, X_val, y_train, y_val = train_test_split( X, y, test_size=0.1, random_state=4) #Data training 90% dan validasi 10%

# Test Length
print ('Train set:', X_train.shape,  y_train.shape)
print ('Val set:', X_val.shape,  y_val.shape)

In [ ]:
svclassifier = SVC(kernel = 'rbf', gamma = 'auto')
svclassifier.fit(X_train, y_train)

In [ ]:
%%time
# Decision Tree Classifier
#---------------------------------
# Biar gak nyakitin mata atas warning yang tidak perlu
#---------------------------------
import warnings
warnings.filterwarnings('ignore')
#---------------------------------
# Jumlah pohon (i = awal, n = akhir)
i = int(1)
j = int(1)
n = int(11)
k = int(11)
konstanta = 1

#Membuat array (Biar gak error di akhiran)
dtc_score = np.empty((n, konstanta))
dtc_score_test = np.empty((n, konstanta))
b_error = np.empty((k, n))
b_error_test = np.empty((k, n))

# Training dan Validasi
while i < n:
    j = konstanta
    while j < k:
        dtclassifier = DecisionTreeClassifier(max_depth = i)           
        dtclassifier.fit(X_train, y_train)                             #Memasukkan data training ke dalam dtc
        y_dtc_pred_val = dtclassifier.predict(X_val)                   #Melakukan prediksi data validasi oleh dtc
        cm_dtc_val = confusion_matrix(y_val, y_dtc_pred_val)           #Membuat matriks kekalutan data validasi dtc
        cr_dtc_val = classification_report(y_val, y_dtc_pred_val)      #Membuat laporan klasifikasi yang telah dilakukan dtc
        dtc_score[i, :] = int(dtclassifier.score(X_val, y_val) * 100)
        #print("Akurasi Data Validasi dtc" + str(i) + ": {}%".format(int(dtc_score[i])))      #Cetak dalam bentuk persentase

        # Test
        y_dtc_pred_test = dtclassifier.predict(X_test)                   #Melakukan prediksi data tes oleh dtc
        cm_dtc_test = confusion_matrix(y_test, y_dtc_pred_test)           #Membuat matriks kekalutan data tes dtc
        cr_dtc_test = classification_report(y_test, y_dtc_pred_test)      #Membuat laporan klasifikasi yang telah dilakukan dtc
        dtc_score_test[i, :] = int(dtclassifier.score(X_test, y_test) * 100)
        #print("Akurasi Data Test dtc" + str(i) + ": {}%".format(int(dtc_score_test[i])))      #Cetak dalam bentuk persentase
        b_error[j, i] = dtc_score[i]
        b_error_test[j,i] = dtc_score_test[i]
        #print(b_error[j,i])
        j += konstanta
    i += konstanta
    
#Figure 1 Grafik validasi
plt.figure()
x = np.arange(konstanta, n-1)
y1 = dtc_score[np.arange(konstanta, n-1)]
y2 = dtc_score_test[np.arange(konstanta, n-1)]
e1 = np.empty((n, konstanta))
e2 = np.empty((n, konstanta))
index1 = konstanta
index2 = n
while index1 < index2:
    e1[index1] = np.std(b_error[np.arange(konstanta,n),index1])
    e2[index1] = np.std(b_error_test[np.arange(konstanta,n),index1])
    index1 += konstanta
    
print("Akurasi Data Validasi dtc: {}%".format(y1))
print("Error Validasi: {}%".format(e1[x]))
plt.axis([0, 11, 0, 110])
plt.errorbar(x, y1, e1[x], color='green', marker='^')
#plt.errorbar(x, y2, e2[x], color='red', marker='^', linestyle='None')
plt.xlabel('Jumlah Cabang')
plt.ylabel('Akurasi')
plt.title('Akurasi Data Validasi dan Test DTC Terhadap Jumlah Cabang')
plt.grid()
plt.show()

plt.figure()
x = np.arange(konstanta, n-1)
y1 = dtc_score[np.arange(konstanta, n-1)]
y2 = dtc_score_test[np.arange(konstanta, n-1)]
e1 = np.empty((n, konstanta))
e2 = np.empty((n, konstanta))
index1 = konstanta
index2 = n
while index1 < index2:
    e1[index1] = np.std(b_error[np.arange(konstanta,n),index1])
    e2[index1] = np.std(b_error_test[np.arange(konstanta,n),index1])
    index1 += konstanta

print("Akurasi Data Test dtc: {}%".format(y2))      #Cetak dalam bentuk persentase
print("Error Test: {}%".format(e2[x]))
plt.axis([0, 11, 0, 110])

plt.errorbar(x, y2, e2[x], color='red', marker='^')
plt.xlabel('Jumlah Cabang')
plt.ylabel('Akurasi')
plt.title('Akurasi Data Validasi dan Test DTC Terhadap Jumlah Cabang')
plt.grid()
plt.show()

In [ ]:
# Confusion matrix (kekalutan) hanya membaca data terakhir saja, respectively
plot_confusion_matrix(cm_dtc_test, classes = classes)
print("DATA TEST DTC")
print(cr_dtc_test)